In [1]:
import os
import glob
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('max_rows', 200)
pd.set_option('max_columns', 200)

perc = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import datetime

date_list = ['20200917', '20200918', '20200921', '20200922', '20200923', '20200924', 
             '20200925', '20200928', '20200929', '20200930']
re = []   
for d in date_list:
    startDate = d
    endDate = d

    readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
    dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
    dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
    rawOrderLog = []
    for thisDate, thisPath in zip(dateLs, dataPathLs):
        data = pd.read_csv(thisPath)
        data = data.rename(columns={'mdClockAtArrival': 'caamd'})
        rawOrderLog += [data]
    rawOrderLog = pd.concat(rawOrderLog, sort=False)

    # rawOrderLog = pd.read_pickle(r'A:\temp\speedCheck\zt96_statwo_20200818.pkl')


    for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
                'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
        rawOrderLog[col] = rawOrderLog[col].astype('int64')   
    rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

    rawOrderLog['clock'] = rawOrderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    rawOrderLog['broker'] = rawOrderLog['accCode'] // 100
    rawOrderLog["broker"] = np.where(rawOrderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), rawOrderLog['accCode'] // 10000, rawOrderLog["broker"])
    rawOrderLog['colo_broker'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['broker'].astype('str')
    rawOrderLog['order'] = rawOrderLog.groupby(['date', 'accCode', 'secid', 'vai']).grouper.group_info[0]
    rawOrderLog['group'] = rawOrderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
    rawOrderLog['startClock'] = rawOrderLog.groupby(['order'])['clockAtArrival'].transform('first')
    rawOrderLog['duration'] = rawOrderLog['clockAtArrival'] - rawOrderLog['startClock']
    rawOrderLog['orderPrice'] = rawOrderLog['orderPrice'].apply(lambda x: round(x, 2))
    rawOrderLog['tradePrice'] = rawOrderLog['tradePrice'].apply(lambda x: round(x, 2))
    rawOrderLog['orderDirection1'] = np.where(rawOrderLog["orderDirection"] == -2, -1, np.where(
        rawOrderLog["orderDirection"] == 2, 1, rawOrderLog["orderDirection"]))
    orderLog = rawOrderLog.copy()

    ### Assertion 1:  make sure same direction in same date, secid, vai
    print('=======================================================================================')
    print('1. same date, secid, vai: same direction')
    orderLog['directNum'] = orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].transform('nunique')
    if len(orderLog[orderLog['directNum'] != 1]) > 0:
        print('opposite direction for same date, same secid, same vai')
        display(orderLog[(orderLog['directNum'] != 1) & (orderLog['updateType'] == 0)][['date', 'accCode', 'secid', 'vai', 'orderDirection', 'order']])
        orderLog = orderLog[orderLog['directNum'] == 1]

    assert((orderLog.groupby(['date', 'secid', 'vai'])['orderDirection1'].nunique() == 1).all() == True)

    ## Assertion 2:  make sure each account, secid, vai only has one insertion
    print('=======================================================================================')
    print('2. same date, secid, vai, accCode: one insertion')
    a = orderLog[orderLog['updateType'] == 0].groupby(['date', 'accCode', 'secid', 'vai', 'order'])['clockAtArrival'].count()
    if len(a[a > 1]) > 0:
        print('more than one insertion at same time')
        a = a[a>1].reset_index()
        display(a)
        orderLog = orderLog[~(orderLog['order'].isin(a['order'].unique()))]

    orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                                 np.where(orderLog['mse'] == 100, 1, 0), np.nan)
    orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()

    placeSZE = orderLog[(orderLog['secid'] >= 2000000) & (orderLog['updateType'] == 0)]
    print('%.2f%% SZE orders triggered by msg data'%(placeSZE[placeSZE['isMsg'] == 1].shape[0]/placeSZE.shape[0]*100))


    ### Assertion 3:  check IPO stocks selling status
    print('=======================================================================================')
    print('3. IPO stocks selling (ars = 301, 302)')
    if orderLog[orderLog['ars'].isin([301, 302])].shape[0] != 0:
        kk = orderLog[orderLog['ars'].isin([301, 302])]
        print(kk)
        try:
            assert(kk[kk['orderDirection1'] == 1].shape[0] == 0)
            print('we only sell, never buy')
        except:
            print('There are IPO buy side orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk[kk['orderDirection1'] == 1])
        kk1 = kk[kk['updateType'] == 0]
        kk1 = kk1.sort_values(by=['accCode', 'secid','clockAtArrival'])
        kk1['diff'] = kk1.groupby(['accCode', 'secid'])['clockAtArrival'].apply(lambda x: x-x.shift(1))
        kk1['diff'] = kk1['diff'].fillna(0)
        try:
            assert(kk1[kk1['diff'] < 10e6].shape[0] == 0)
            print('for each stock in the same account, there is no insertion within 10 seconds of the previous insertion')
        except:
            print('There are insertion within 10 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk1[kk1['diff'] < 10e6])
        kk2 = kk[(kk['updateType'] == 1)]
        try:
            assert(kk2[kk2['duration'] < 3e6].shape[0] == 0)
            print('for each stock in the same account, the cancellation of an order happens more than 3 seconds after the insertion')
        except:
            print('There are cancellation within 3 seconds for orders under same account same stock!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
            print(kk2[kk2['duration'] < 3e6])


    ### Assertion 4: check updateType == 7 orders, make sure updateType == 7 orders < 20 per account, < 100 in total
    print('=======================================================================================')
    print('4. updateType 7 orders')
    if orderLog[orderLog['updateType'] == 7].shape[0] != 0:
        assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['order'].nunique().max() < 20)
        assert(orderLog[orderLog['updateType'] == 7].groupby('accCode')['order'].nunique().sum() < 100)

    ### Assertion 5: check updateType == 6 orders, make sure updateType == 6 orders < 5% per account
    print('=======================================================================================')
    print('5. updateType 6 orders')
    k1 = orderLog[orderLog['updateType'] == 6].groupby('accCode')['order'].nunique().reset_index()
    k2 = orderLog.groupby('accCode')['order'].nunique().reset_index()
    k = pd.merge(k1, k2, on='accCode', how='left')
    k['prob'] = k['order_x']/k['order_y']
    try:
        assert(sum(k['prob'] >= 0.05) == 0)
    except:
        print('There are accounts with more than 5% updateType 6 orders!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
        print(k[k['prob'] >= 0.05])

    ### Assertion 6: check CYB orders, make sure CYB stocks total absOrderSize < 30w
    print('=======================================================================================')
    print('6. CYB stocks total order size < 30w')
    try:
        assert(orderLog[(orderLog['secid'] >= 2300000) & (orderLog['updateType'] == 0)]['absOrderSize'].max() <= 300000)
    except:
        print('CYB stocks total absOrderSize >= 30w!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')


    ### Assertion 7:  make sure there is no unexpected updateType 
    print('=======================================================================================')
    print('7. unexpected updateType')
    def getTuple(x):
        return tuple(i for i in x)

    checkLog = orderLog[~((orderLog['updateType'] == 4) & (orderLog.groupby(['order'])['updateType'].shift(-1) == 4))]
    checkLog = checkLog.groupby(['order'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
    checkLog['status'] = np.where(checkLog['updateType'].isin([(0, 2, 4), (0, 2, 1, 4), (0, 2, 1, 2, 4), (0, 2, 4, 1, 4), (0, 4), (0, 1, 4), (0, 4, 1, 4), (0, 2, 2, 4), (0, 4, 2, 4), (0, 2, 2, 1, 4), (0, 2, 2, 4, 1, 4)]),0,
                         np.where(checkLog['updateType'].isin([(0, 2, 4, 1, 3), (0, 2, 4, 1, 4, 3), (0, 2, 1, 4, 3), (0, 4, 1, 3), (0, 1, 4, 3),
                                                                   (0, 2, 2, 4, 1, 3), (0, 2, 2, 4, 1, 4, 3), (0, 2, 2, 1, 4, 3), (0, 4, 2, 4, 1, 3),
                                                                   (0, 4, 2, 1, 3), (0, 4, 1, 4, 3), (0, 4, 1)]), 1,
                         np.where(checkLog['updateType'].isin([(0, 2, 1, 3), (0, 2, 2, 1, 3), (0, 2, 3), (0, 3), (0, 1, 3), (0, ), (0, 2), (0, 2, 1), (0, 2, 2)]), 2, 3)))

    orderLog = pd.merge(orderLog, checkLog[['order', 'status']], how='left', on=['order'], validate='many_to_one')
    orderLog = orderLog[orderLog['status'].isin([0, 1, 2])].reset_index(drop=True)

    ### Assertion 8:  make sure status==0 got all traded
    print('=======================================================================================')
    print('8. status == 0: all traded')
    a = orderLog[orderLog['status'] == 0]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    print('in total trade, any fill != total cases')
    display(a[a['filled'] != a['total']])
    if a[a['filled'] != a['total']].shape[0] > 0:
        removeOrderLs = a[a['filled'] != a['total']]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### Assertion 9:  make sure status==1 got partial traded
    print('=======================================================================================')
    print('9. status == 1: partial traded')
    a = orderLog[orderLog['status'] == 1]
    a = a.groupby(['order'])[['absOrderSizeCumFilled', 'absOrderSize']].max().reset_index()
    a.columns = ['order', 'filled', 'total']
    print('in partial trade, any fill >= total or fill is 0 cases for updateType 4')
    display(a[(a['filled'] >= a['total']) | (a['filled'] == 0)])
    if a[(a['filled'] >= a['total']) | (a['filled'] == 0)].shape[0] > 0:
        removeOrderLs = a[(a['filled'] >= a['total']) | (a['filled'] == 0)]['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    ### Assertion 10: make sure no cancellation within 1 sec
    print('=======================================================================================')
    print('10. no cancellation within 1 sec')
    a = orderLog[(orderLog['updateType'] == 1) & (orderLog['duration'] < 1e6)]
    print('any cancellation within 1 sec')
    display(a)
    if a.shape[0] > 0:
        removeOrderLs = a['order'].unique()
        orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]


    ### Assertion 11: make sure no order has shares > 80w or notional > 800w
    print('=======================================================================================')
    print('11. Orders with size > 80w or notional > 800w')
    orderLog['orderNtl'] = orderLog['absOrderSize'] * orderLog['orderPrice']
    if orderLog[orderLog['absOrderSize'] > 800000].shape[0] > 0:
        print('some order quantity are > 80w')
        print(orderLog[orderLog['absOrderSize'] > 800000].groupby(['colo', 'accCode'])['order'].nunique())
        display(orderLog[orderLog['absOrderSize'] > 800000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                             'orderNtl', 'orderDirection', 'clock', 'order']])

    if orderLog[orderLog['orderNtl'] > 8000000].shape[0] > 0:
        print('some order ntl are > 800w')
        print(orderLog[orderLog['orderNtl'] > 8000000].groupby(['colo', 'accCode'])['order'].nunique())
        display(orderLog[orderLog['orderNtl'] > 8000000][['date', 'accCode', 'secid', 'vai', 'absOrderSize', 'orderPrice',
                                                          'orderNtl', 'orderDirection', 'clock', 'order', "updateType", 
                                                          "tradePrice", "absOrderSizeCumFilled", "absFilledThisUpdate"]])

    removeOrderLs = list(set(orderLog[orderLog['absOrderSize'] > 800000]['order'].unique()) | set(orderLog[orderLog['orderNtl'] > 8000000]['order'].unique()))
    orderLog = orderLog[~(orderLog['order'].isin(removeOrderLs))]

    orderLog = orderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

    orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
    orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
    orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
    orderLog['tradeNtl'] = orderLog.groupby('order')['tradeNtl'].transform('sum')
    orderLog["ars"] = orderLog.groupby(['order'])['ars'].transform('first')
    orderLog['sta'] = np.where(orderLog['ars'].isin([121, 221, 321, 131, 231, 331]), 'statwo', 'staone')
    orderLog['cumFillSize'] = orderLog.groupby(['order'])['absOrderSizeCumFilled'].transform('max')
    orderLog['status'] = np.where(orderLog['cumFillSize'] == 0, '3. no filled', 
                                    np.where(orderLog['cumFillSize'] < orderLog['absOrderSize'], '2. partial filled', '1. full filled'))


    startDate = d
    endDate = d
    readPath = r'L:\orderLog\result\marketPos'
    dataPathLs = np.array(glob.glob(os.path.join(readPath, 'SZspeed_***.pkl')))
    dateLs = np.array([os.path.basename(i).split('.')[0].split('_')[1] for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
    checkData = []
    for path in dataPathLs:
        data = pd.read_pickle(path)
        checkData += [data]
    checkLog = pd.concat(checkData).reset_index(drop=True)   
    checkLog['date'].unique()
    
    
    checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
    checkLog = checkLog[~checkLog['accCode'].isnull()]
    targetStockSZ = checkLog['SecurityID'].unique()
    
    startDate = d
    endDate = d

    readPath = r'\\192.168.10.30\Kevin_zhenyu\orderLog\equityTradeLogs'
    dataPathLs = np.array(glob.glob(os.path.join(readPath, 'speedCompare***.csv')))
    dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]
    dateLs = dateLs[(dateLs >= startDate) & (dateLs <= endDate)]
    OrderLog = []
    TradeLog = []
    for date in dateLs:
        readPath = 'A:\\rawData'
        mdOrderLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdOrderLog***.csv'))[-1]
        OrderLog1 = pd.read_csv(mdOrderLogPath)
        OrderLog1 = OrderLog1[OrderLog1['SecurityID'].isin(targetStockSZ)]
        OrderLog1['OrderType'] = OrderLog1['OrderType'].astype(str)
        OrderLog1 = OrderLog1[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'Side',
                                 'OrderType', 'Price', 'OrderQty']]
        OrderLog1['date'] = int(date)
        OrderLog += [OrderLog1]
        del OrderLog1


        mdTradeLogPath = glob.glob(os.path.join(readPath, 'logs_%s_zs_92_01***'%date, 'mdTradeLog***.csv'))[-1]
        TradeLog1 = pd.read_csv(mdTradeLogPath, encoding='utf-8')
        TradeLog1['ExecType'] = TradeLog1['ExecType'].astype(str)
        TradeLog1 = TradeLog1[TradeLog1['SecurityID'].isin(targetStockSZ)]
        TradeLog1 = TradeLog1[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum',
                                 'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']]
        TradeLog1['date'] = int(date)
        TradeLog1 = TradeLog1[TradeLog1['ExecType'] == '4']
        TradeLog += [TradeLog1]
        del TradeLog1
    OrderLog = pd.concat(OrderLog).reset_index(drop=True)
    TradeLog = pd.concat(TradeLog).reset_index(drop=True)
    checkLog = checkLog[['date', 'SecurityID', 'sequenceNo', 'colo', 'accCode', 'secid', 'ApplSeqNum', 'Side']]
    checkLog = pd.merge(checkLog, OrderLog[['SecurityID', 'date', 'sequenceNo', 'ApplSeqNum']],
                 on=['date', 'SecurityID', 'sequenceNo'], validate='one_to_one')
    checkLog = checkLog.rename(columns={'ApplSeqNum_y':'ApplSeqNum_order', 'ApplSeqNum_x':'ApplSeqNum'})
    checkLog = pd.merge(checkLog, OrderLog[['SecurityID', 'date', 'ApplSeqNum', 'clockAtArrival']],
                 on=['date', 'SecurityID', 'ApplSeqNum'], validate='many_to_one')
    checkLog = checkLog.rename(columns={'clockAtArrival':'market_insert'})
    del OrderLog
    
    orderLog['ApplSeqNum'] = orderLog.groupby(['order'])['ApplSeqNum'].transform('first')
    orderLog['caamd'] = orderLog.groupby(['order'])['caamd'].transform('first')
    checkLog = pd.merge(checkLog, orderLog[(orderLog['exchange'] == 'SZE')
                                    & (orderLog['ApplSeqNum'] != 0)][['date', 'colo', 'accCode', 'secid', 'ApplSeqNum', 'clockAtArrival', 'caamd', 'updateType', 'order']],
                 on=['date', 'colo', 'accCode', 'secid', 'ApplSeqNum'], validate='one_to_many')
    checkLog = checkLog.rename(columns={'clockAtArrival':'internal_trade', 'caamd':'internal_insert'})
    checkLog = checkLog[checkLog['updateType'] == 3]
    checkLog1 = checkLog[checkLog['Side'] == 1]
    checkLog2 = checkLog[checkLog['Side'] == 2]
    
    checkLog1 = pd.merge(checkLog1, TradeLog[['SecurityID', 'date', 'clockAtArrival', 'BidApplSeqNum', 'sequenceNo']],
             left_on=['date', 'SecurityID', 'ApplSeqNum_order'], right_on=['date', 'SecurityID', 'BidApplSeqNum'], validate='many_to_many')
    checkLog1 = checkLog1.rename(columns={'clockAtArrival':"market_trade", 'sequenceNo_x':'sequenceNo_order','sequenceNo_y':'sequenceNo_trade'})
    checkLog1 = checkLog1.sort_values(by=['date', 'secid', 'ApplSeqNum_order', 'internal_trade'])
    checkLog1['count1'] = checkLog1.groupby(['date', 'secid', 'ApplSeqNum_order', 'internal_trade']).cumcount()
    checkLog1 = checkLog1.sort_values(by=['date', 'secid', 'ApplSeqNum_order', 'sequenceNo_trade'])
    checkLog1['count2'] = checkLog1.groupby(['date', 'secid', 'ApplSeqNum_order', 'sequenceNo_trade']).cumcount()
    c1 = checkLog1.groupby(['date', 'secid', 'ApplSeqNum_order'])['count1'].max().reset_index()
    c2 = checkLog1.groupby(['date', 'secid', 'ApplSeqNum_order'])['count2'].max().reset_index()
    cc = pd.merge(c1, c2, on=['date', 'secid', 'ApplSeqNum_order'])
    assert(cc[cc['count1'] != cc['count2']].shape[0] == 0)
    checkLog1 = checkLog1[checkLog1['count1'] == checkLog1['count2']]
    
    checkLog2 = pd.merge(checkLog2, TradeLog[['SecurityID', 'date', 'clockAtArrival', 'OfferApplSeqNum', 'sequenceNo']],
             left_on=['date', 'SecurityID', 'ApplSeqNum_order'], right_on=['date', 'SecurityID', 'OfferApplSeqNum'], validate='many_to_many')
    del TradeLog
    checkLog2 = checkLog2.rename(columns={'clockAtArrival':"market_trade", 'sequenceNo_x':'sequenceNo_order','sequenceNo_y':'sequenceNo_trade'})
    checkLog2 = checkLog2.sort_values(by=['date', 'secid', 'ApplSeqNum_order', 'internal_trade'])
    checkLog2['count1'] = checkLog2.groupby(['date', 'secid', 'ApplSeqNum_order', 'internal_trade']).cumcount()
    checkLog2 = checkLog2.sort_values(by=['date', 'secid', 'ApplSeqNum_order', 'sequenceNo_trade'])
    checkLog2['count2'] = checkLog2.groupby(['date', 'secid', 'ApplSeqNum_order', 'sequenceNo_trade']).cumcount()
    c1 = checkLog2.groupby(['date', 'secid', 'ApplSeqNum_order'])['count1'].max().reset_index()
    c2 = checkLog2.groupby(['date', 'secid', 'ApplSeqNum_order'])['count2'].max().reset_index()
    cc = pd.merge(c1, c2, on=['date', 'secid', 'ApplSeqNum_order'])
    assert(cc[cc['count1'] != cc['count2']].shape[0] == 0)
    checkLog2 = checkLog2[checkLog2['count1'] == checkLog2['count2']]
    
    checkLog = pd.concat([checkLog1, checkLog2])
    checkLog['interval1'] = checkLog['internal_trade'] - checkLog['internal_insert']
    checkLog['interval2'] = checkLog['market_trade'] - checkLog['market_insert']
    checkLog['diff'] = checkLog['interval1'] - checkLog['interval2']
    
    re += [checkLog]
    print(d)
re = pd.concat(re).reset_index(drop=True)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


1. same date, secid, vai: same direction
opposite direction for same date, same secid, same vai


,date,accCode,secid,vai,orderDirection,order
70141,20200917,9741,1603090,417200,1,51208
70145,20200917,9741,1603090,417200,-1,51208
70148,20200917,975602,1603090,417200,1,79100
70152,20200917,975602,1603090,417200,-1,79100
103449,20200917,9248,1603757,1898460,-1,44836
103453,20200917,527701,1603757,1898460,1,70108
103456,20200917,529101,1603757,1898460,1,75479
146425,20200917,6683,2002022,5480190,1,33377
146429,20200917,9685,2002022,5480190,-1,49723


2. same date, secid, vai, accCode: one insertion
more than one insertion at same time


,date,accCode,secid,vai,order,clockAtArrival
0,20200917,5273,2002536,4000,6189,2
1,20200917,6683,2002791,858006,34112,2
2,20200917,6683,2300785,645598,35260,2
3,20200917,8854,2300776,370012,37465,2
4,20200917,9551,1603039,519005,48753,2
5,20200917,9741,1603016,340200,51081,2
6,20200917,9741,1603139,329900,51275,3
7,20200917,9741,1603860,401500,51811,2
8,20200917,9741,2002896,1105700,52746,2
9,20200917,9741,2300374,698520,53170,2


...

,Unnamed: 0,clockAtArrival,caamd,secid,updateType,vai,ars,absFilledThisUpdate,orderDirection,absOrderSize,...,broker,colo_broker,order,group,startClock,duration,orderDirection1,directNum,isMsg,status


11. Orders with size > 80w or notional > 800w


C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


20200930


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:322: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [2]:
re['diff'].describe([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype(np.int64)

count         59409
mean         -44908
std        12454598
min     -3017972141
1%            -2158
5%             -461
10%            -214
20%             -90
30%             -21
40%              36
50%              92
60%             152
70%             236
80%             663
90%            3662
95%           25222
99%           54743
max        40100252
Name: diff, dtype: int64

In [3]:
from IPython.display import display, HTML
re['accCode'] = re['accCode'].astype(int)
HTML(re.groupby(['colo'])['diff'].describe().sort_values(by='mean').astype(int).to_html())

,count,mean,std,min,25%,50%,75%,max
colo,,,,,,,,
zs_94_04,1164,-2558560,88460120,-2147483648,26114,29406,41214,4820574
zt_52_05,331,-50225,344591,-6111465,-37682,-29952,-21198,332014
zs_52_06,12997,-23790,2748701,-313137318,58,156,265,5388344
zt_96_01,108,-1727,2349,-12733,-2561,-1698,-905,3758
zs_94_03,216,-1075,1075,-4644,-1320,-701,-314,113
zs_64_01,1456,-583,513,-3701,-701,-502,-337,4640
zs_92_02,370,-562,585,-6526,-777,-371,-244,9
zs_92_01,161,-367,307,-1126,-663,-247,-123,147
zs_54_01,2777,-162,464,-3129,-342,-121,20,9154
